In [5]:
#https://sbert.net/
#https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
#https://huggingface.co/openai/whisper-large-v3

In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np

/Users/brandonamarasingam/anaconda3/envs/pytorch/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
import json

# Load the list from the JSON file
with open('speech_total.json', 'r') as file:
    speech_total = json.load(file)

len(speech_total)

828

In [3]:
# Load the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Your text data
texts = ["Your first text here", "Your second text here", "And so on..."]

# Generate embeddings
embeddings = model.encode(texts)

# Now embeddings is a numpy array of shape (n_texts, embedding_dim)
print(embeddings.shape)

# You can now use these embeddings for further processing or storage

/Users/brandonamarasingam/anaconda3/envs/pytorch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


(3, 384)


In [4]:
#https://pypi.org/project/faiss-cpu/

In [6]:
import numpy as np
import faiss

# Assuming you have your embeddings in a variable called 'embeddings'
# and your original texts in a list called 'texts'

# Convert embeddings to a numpy array if it's not already
embeddings_np = np.array(embeddings).astype('float32')

# Get the dimension of the embeddings
dimension = embeddings_np.shape[1]

# Create a FAISS index
index = faiss.IndexFlatL2(dimension)

# Add the vectors to the index
index.add(embeddings_np)

# Optionally, you can save the index to a file
faiss.write_index(index, "faiss_index.bin")

# To demonstrate how to use the index, let's do a simple search
# Let's say we want to find the 2 nearest neighbors for the first embedding
k = 2
D, I = index.search(embeddings_np[:1], k)

print(f"Distances: {D}")
print(f"Indices: {I}")

# Print the original texts for the nearest neighbors
for idx in I[0]:
    print(f"Nearest neighbor text: {texts[idx]}")

# If you need to load the index later, you can use:
# loaded_index = faiss.read_index("faiss_index.bin")

Distances: [[0.        0.3136469]]
Indices: [[0 1]]
Nearest neighbor text: Your first text here
Nearest neighbor text: Your second text here


In [7]:
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import faiss

# Load the embedding model
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Load the language model and tokenizer
model_name = "distilgpt2"  # A smaller model that can run on CPU
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create a text generation pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Load your FAISS index (assuming you've already created it)
index = faiss.read_index("faiss_index.bin")

# Load your original texts (assuming you've saved them somewhere)
import pickle
with open('original_texts.pkl', 'rb') as f:
    texts = pickle.load(f)

def retrieve_context(query, k=3):
    # Generate embedding for the query
    query_embedding = embed_model.encode([query])[0]
    
    # Search the FAISS index
    D, I = index.search(np.array([query_embedding]).astype('float32'), k)
    
    # Retrieve the corresponding texts
    retrieved_texts = [texts[i] for i in I[0]]
    
    return " ".join(retrieved_texts)

def generate_response(query, max_length=100):
    # Retrieve relevant context
    context = retrieve_context(query)
    
    # Create a prompt
    prompt = f"Context: {context}\n\nQuery: {query}\n\nResponse:"
    
    # Generate response
    response = generator(prompt, max_length=max_length, num_return_sequences=1)[0]['generated_text']
    
    return response.split("Response:")[-1].strip()

# Example usage
query = "What are Elon Musk's thoughts on AI?"
response = generate_response(query)
print(f"Query: {query}")
print(f"Response: {response}")

/Users/brandonamarasingam/anaconda3/envs/pytorch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


FileNotFoundError: [Errno 2] No such file or directory: 'original_texts.pkl'